Set initial folder locations, as well as the number of positions (for running array jobs)

In [3]:
n_pos = 30
job_path = '/path/to/job/logging/details'
config_path = '/path/to/experiment/analysis/exp_config.yml'
image_path = '/path/to/experiment/images'

Convert to OME-ZARR from ND2 format:

In [ ]:
params = {}
params['job_path'] =job_path+'convert'
params['job_name'] = 'convert'
params['time'] = '60:00'
params['cpus_per_task'] = '2'
params['mem_per_cpu'] = '4G'
params['env'] = '~/env/looptrace'
params['bin_path'] = '~/looptrace/bin/cli/convert_datasets_to_zarr.py'
params['array'] = '0-'+str(n_pos-1)
params['partition'] = 'htc-el8'
params['run'] = ''

options = {}
options['input_folders'] = '/path/to/first_image_folder /path/to/second_image_folder'   #Multiple folders will be concatenated along their time-axis, and must match along other dimensions. Any number of image folders containing ND2 images of the format Time00000_Point0000_Channel555 nm,635 nm_Seq0000.nd2 (Nikon JOBS output) should work.
options['output_folder'] = '/path/to/output/zarr_image'
options['n_pos'] = str(n_pos)

cmd = ' '.join(['python3','~/looptrace/bin/cli/submit_cluster_job.py'] + [str('--'+k+' '+params[k]) for k in params] + [' --options "'] + [str('--'+k+' '+options[k]) for k in options] + ['"'])
print(cmd)
!{cmd}

Run PSF extraction prior to deconvolution:

In [ ]:
options = {}

options['config_path'] = config_path
options['image_path'] = image_path

bin_path = '~/looptrace/bin/cli/extract_exp_psf.py'

cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Run deconvolution on HPC:

In [ ]:
params = {}
params['job_path'] =job_path+'decon'
params['job_name'] = 'decon_cluster'
params['time'] = '60:00'
params['cpus_per_task'] = '2'
params['mem_per_cpu'] = '4G'
params['env'] = '~/env/looptrace'
params['bin_path'] = '~/looptrace/bin/cli/decon.py'
params['array'] = '0-'+str(n_pos-1)
params['partition'] = 'gpu-el8'
params['gres'] = 'gpu:1'
params['module'] = 'TensorFlow/2.6.0-foss-2021a-CUDA-11.3.1'
params['run'] = ''

options = {}
options['config_path'] = config_path
options['image_path'] = image_path

cmd = ' '.join(['python3','~/looptrace/bin/cli/submit_cluster_job.py'] + [str('--'+k+' '+params[k]) for k in params] + [' --options "'] + [str('--'+k+' '+options[k]) for k in options] + ['"'])
print(cmd)
!{cmd}

Save single nuclei images:

In [ ]:
options = {}

options['config_path'] = config_path
options['image_path'] = image_path

bin_path = '~/looptrace/bin/cli/nuc_image_save.py'

cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Segment nuclei using CellPose for 2D nuclei images on local machine (if running in 3D, should do this as cluster job using the syntax from the other jobs):

In [ ]:
options = {}

options['config_path'] = config_path
options['image_path'] = image_path

bin_path = '~/looptrace/bin/cli/nuc_label.py'

cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Run drift correction as a cluster array job:

In [ ]:
params = {}
params['job_path'] = job_path+'dc'
params['job_name'] = 'dc_cluster'
params['time'] = '30:00'
params['cpus_per_task'] = '2'
params['mem_per_cpu'] = '4G'
params['env'] = '~/env/looptrace'
params['bin_path'] = '~/looptrace/bin/cli/drift_correct.py'
params['partition'] = 'htc-el8'
params['array'] = '0-'+str(n_pos-1)
params['run'] = ''

options = {}
options['config_path'] = config_path
options['image_path'] = image_path

cmd = ' '.join(['python3','~/looptrace/bin/cli/submit_cluster_job.py'] + [str('--'+k+' '+params[k]) for k in params] + [' --options "'] + [str('--'+k+' '+options[k]) for k in options] + ['"'])
print(cmd)
!{cmd}

Merging of the separate drift correction files:

In [ ]:
bin_path = '~/looptrace/bin/cli/cluster_analysis_cleanup.py'
options = {}
options['config_path'] = config_path
cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Run spot detection according to config file:

In [ ]:
params = {}
params['job_path'] = job_path+'dc'
params['job_name'] = 'dc_cluster'
params['time'] = '10:00'
params['cpus_per_task'] = '2'
params['mem_per_cpu'] = '4G'
params['env'] = '~/env/looptrace'
params['bin_path'] = '~/looptrace/bin/cli/detect_spots.py'
params['partition'] = 'htc-el8'
params['array'] = '0-'+str(n_pos-1)
params['run'] = ''

options = {}
options['config_path'] = config_path
options['image_path'] = image_path

cmd = ' '.join(['python3','~/looptrace/bin/cli/submit_cluster_job.py'] + [str('--'+k+' '+params[k]) for k in params] + [' --options "'] + [str('--'+k+' '+options[k]) for k in options] + ['"'])
print(cmd)
!{cmd}

Clean up tables of detected spots:

In [ ]:
bin_path = '~/looptrace/bin/cli/cluster_analysis_cleanup.py'
options = {}
options['config_path'] = config_path
cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Generate full table of drift corrected spot bounding boxes:

In [ ]:
options = {}

options['config_path'] = config_path
options['image_path'] = image_path

bin_path = '~/looptrace/bin/cli/extract_spots_table.py'

cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Extract spot images from full dataset:

In [ ]:
params = {}
params['job_path'] = job_path+'dc'
params['job_name'] = 'dc_cluster'
params['time'] = '10:00'
params['cpus_per_task'] = '2'
params['mem_per_cpu'] = '4G'
params['env'] = '~/env/looptrace'
params['bin_path'] = '~/looptrace/bin/cli/extract_spots.py'
params['partition'] = 'htc-el8'
params['array'] = '0-'+str(n_pos-1)
params['run'] = ''

options = {}
options['config_path'] = config_path
options['image_path'] = image_path

cmd = ' '.join(['python3','~/looptrace/bin/cli/submit_cluster_job.py'] + [str('--'+k+' '+params[k]) for k in params] + [' --options "'] + [str('--'+k+' '+options[k]) for k in options] + ['"'])
print(cmd)
!{cmd}

Clean up extracted spots images:

In [ ]:
bin_path = '~/looptrace/bin/cli/extract_spots_cluster_cleanup.py'
options = {}
options['config_path'] = config_path
cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

Run tracing workflow:

In [ ]:
params = {}
params['job_path'] = job_path+'dc'
params['job_name'] = 'dc_cluster'
params['time'] = '10:00'
params['cpus_per_task'] = '2'
params['mem_per_cpu'] = '4G'
params['env'] = '~/env/looptrace'
params['bin_path'] = '~/looptrace/bin/cli/tracing.py'
params['partition'] = 'htc-el8'
params['array'] = '0-'+str(n_pos-1)
params['run'] = ''

options = {}
options['config_path'] = config_path
options['image_path'] = image_path

cmd = ' '.join(['python3','~/looptrace/bin/cli/submit_cluster_job.py'] + [str('--'+k+' '+params[k]) for k in params] + [' --options "'] + [str('--'+k+' '+options[k]) for k in options] + ['"'])
print(cmd)
!{cmd}

Clean up tracing tables:

In [ ]:
bin_path = '~/looptrace/bin/cli/cluster_analysis_cleanup.py'
options = {}
options['config_path'] = config_path
cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
print(cmd)
!{cmd}

(Optional) This adds the nuclei label (and classification of nuc_class folder with classified nuclei exists) to ROI and traces table:

options = {}

options['config_path'] = config_path
options['image_path'] = image_path

bin_path = '~/looptrace/bin/cli/assign_spots_to_nucs.py'

cmd = ' '.join(['python3 ']+[bin_path]+[str('--'+k+' '+options[k]) for k in options])
!{cmd}